# 1. Introduction
xxxx

In [13]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import nltk

from tqdm import tqdm
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


sns.set()
random_state = 1

# 2. Data Preprocessing
We will be using two datasets to aid in our stock prediction.
1. Apple Stock Prices (aapl_stock_raw.csv)
2. Twitter Tweets (tweet.csv) *

* The original Tweets dataset is too large to be pushed onto GitHub. 

## 2a. Obtain Data
The dataset we will be using for this problem will consist of the historical daily stock prices of AAPL from 2015-01-01 to 2019-12-31. The description of the data features are as follows:
- date = Date of trading day
- high = Maximum traded price of AAPL stock on that day
- low = Minimum traded price of AAPL stock on that day
- open = The first traded price of AAPL stock on that day
- close = The last traded price of AAPL stock on that day
- adjclose = Adjusted closing price of the day, after accounting for all applicable splits and dividend distributions
- volume = Number of shares of AAPL stock traded on that day
- ticker = AAPL stock symbol

In [57]:
start_date = '2015-01-01'
end_date = '2019-12-31'

# Import dataset
df = pd.read_csv("aapl_stock_raw.csv", index_col=0)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
# df.drop(columns=["adjclose", "volume", "ticker"], inplace=True)
df.drop(columns=["ticker"], inplace=True)

# Remove entries outside daterange
df = df[start_date:end_date]

df['change'] = df['close'] - df['open']


fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()
df

,open,high,low,close,adjclose,volume,change
date,,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.565697,212818400,-0.514999
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.873650,257142000,-0.510000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.875887,263188400,-0.070000
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.210684,160423600,0.137501
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.140913,237458000,0.664999
...,...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.517097,48478800,-0.105003
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.896324,93121200,1.272499
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.869415,146266000,-0.330002


In [58]:
tweets = pd.read_csv(f"Tweet.csv")
display(tweets)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
...,...,...,...,...,...,...,...
3717959,1212159765914079234,TEEELAZER,1577836383,That $SPY $SPX puuump in the last hour was the...,1,0,6
3717960,1212159838882533376,ShortingIsFun,1577836401,In 2020 I may start Tweeting out positive news...,0,0,1
3717961,1212160015332728833,Commuternyc,1577836443,Patiently Waiting for the no twitter sitter tw...,0,0,5
3717962,1212160410692046849,MoriaCrypto,1577836537,I don't discriminate. I own both $aapl and $ms...,1,0,1


### Removing unwanted companies
The tweet dataset shows the tweets that mentions the top NASDAQ companies. Lets remove the unwanted companies.

In [6]:
tweets = tweets[tweets['body'].str.contains('Apple|AAPL|APPLE|apple|aapl')]
display(tweets)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
5,550443808606126081,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
6,550443809700851716,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
...,...,...,...,...,...,...,...
3717957,1212159254884433921,QuantWolfLine,1577836261,Imagine calling your broker-dealer and wanting...,1,0,1
3717958,1212159275637886976,GMGRIFF_79,1577836266,"$AAPL yearly~ Heck of a year.. Jan. 2, 1999~ar...",0,0,1
3717959,1212159765914079234,TEEELAZER,1577836383,That $SPY $SPX puuump in the last hour was the...,1,0,6
3717962,1212160410692046849,MoriaCrypto,1577836537,I don't discriminate. I own both $aapl and $ms...,1,0,1


### Cleaning the headers 
Changing the header so as to be more presentable

In [59]:
clean_tweet = pd.DataFrame(tweets[["tweet_id","writer","post_date","body","comment_num","retweet_num","like_num"]])
dict = {'tweet_id' : 'Tweet ID',
        'writer' : 'Writer',
        'post_date' : 'date',
        'body' : 'Tweet',
        'comment_num' : 'Comments',
        'retweet_num' : 'Retweets',
        'like_num' : 'Likes'
       }
tweet1 = clean_tweet.rename(columns=dict)
tweet1.reset_index(inplace=True,drop=True)
tweet1.head()

,Tweet ID,Writer,date,Tweet,Comments,Retweets,Likes
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


### Cleaning up date  
We further clean the dataset by converting UNIX time to GMT

In [34]:
#Converting unix date to datetime, as well as removing time posted->only show date posted
#Converting GMT+3 to GMT
import datetime
import pytz
tweet1['date'] = tweet1['date'].apply(lambda x: datetime.datetime.fromtimestamp(x, pytz.timezone('Etc/GMT+3')).astimezone(pytz.timezone('Etc/GMT')))
tweet1['date'] = tweet1['date'].dt.date
tweet1 = tweet1.sort_values('date', ascending=True)
display(tweet1)

,Tweet ID,Writer,date,Tweet,Comments,Retweets,Likes
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
206,550736241738526720,TweakTown,2015-01-01,Apple faces lawsuit over massive storage space...,0,0,2
205,550732338670227457,MacHashNews,2015-01-01,"Today's apps gone free: Decide Now, OftenType,...",0,1,3
204,550731345614602240,TalkMarkets,2015-01-01,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1
203,550730566656876548,johnmknox,2015-01-01,Availability & price are a huge problem plagui...,0,0,1
...,...,...,...,...,...,...,...
1496446,1212019899020611586,HottestStockNow,2019-12-31,"Thinking about buying stock in $AAPL, $BBY, $C...",0,0,0
1496447,1212020035910275072,RyanSTTrader,2019-12-31,$AAPL Unreal,0,0,1
1496448,1212020218878296066,seattlehikertoo,2019-12-31,$AAPL march to more nath. 5G coming and everyo...,0,0,0
1496450,1212020375711887362,IchimokuD,2019-12-31,If it is true that everyone already owns Apple...,0,0,0


### Further cleaning 
Tweets usually contains the following:

    1. User Tags (Starts with @)
    
    2. Website Links (Starts with http)
        
To allow for better modelling, we will replace these with generic terms so as to not affect our sentiment score

In [35]:
#Replacing user tags and website links to generic terms

# Define a function to clean the text
def clean_text(text):
    words = text.split()
    for i in range(len(words)):
        if words[i].startswith('@'):
            words[i] = '@user'
        elif words[i].startswith('http'):
            words[i] = 'http'
    return ' '.join(words)

# Apply the clean_text function to the 'text' column
tweet1['Tweet'] = tweet1['Tweet'].apply(clean_text)
# Display the results
display(tweet1)

,Tweet ID,Writer,date,Tweet,Comments,Retweets,Likes
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! http...",0,0,1
206,550736241738526720,TweakTown,2015-01-01,Apple faces lawsuit over massive storage space...,0,0,2
205,550732338670227457,MacHashNews,2015-01-01,"Today's apps gone free: Decide Now, OftenType,...",0,1,3
204,550731345614602240,TalkMarkets,2015-01-01,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1
203,550730566656876548,johnmknox,2015-01-01,Availability & price are a huge problem plagui...,0,0,1
...,...,...,...,...,...,...,...
1496446,1212019899020611586,HottestStockNow,2019-12-31,"Thinking about buying stock in $AAPL, $BBY, $C...",0,0,0
1496447,1212020035910275072,RyanSTTrader,2019-12-31,$AAPL Unreal,0,0,1
1496448,1212020218878296066,seattlehikertoo,2019-12-31,$AAPL march to more nath. 5G coming and everyo...,0,0,0
1496450,1212020375711887362,IchimokuD,2019-12-31,If it is true that everyone already owns Apple...,0,0,0


### Creating Categorical Column

We will add a column 'Label'. A value of 1 means that closing stock price went up from the previous close and 0 means closing stock price went down from previous close

In [60]:
# Shift the 'Close' column by 1 day to get the previous day's prices
df['prev close'] = df['close'].shift(1)

# Compare 'close' with 'Prev Close' to get the labels
df['label'] = (df['close'] > df['prev close']).astype(int)

#AAPL close price before 02/01/2015 is 27.59
df.loc[0, 'label'] = 1
df.head()

,open,high,low,close,adjclose,volume,change,prev close,label
date,,,,,,,,,
2015-01-02 00:00:00,27.847500,27.860001,26.837500,27.332500,24.565697,212818400.0,-0.514999,NaN,0.0
2015-01-05 00:00:00,27.072500,27.162500,26.352501,26.562500,23.873650,257142000.0,-0.510000,27.332500,0.0
2015-01-06 00:00:00,26.635000,26.857500,26.157499,26.565001,23.875887,263188400.0,-0.070000,26.562500,1.0
2015-01-07 00:00:00,26.799999,27.049999,26.674999,26.937500,24.210684,160423600.0,0.137501,26.565001,1.0
2015-01-08 00:00:00,27.307501,28.037500,27.174999,27.972500,25.140913,237458000.0,0.664999,26.937500,1.0


## 2b. Exploratory Data Analysis
Analyse dataset features

In [61]:
df.describe()


,open,high,low,close,adjclose,volume,change,prev close,label
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258.000000,1257.000000,1259.000000
mean,38.599344,38.939050,38.268752,38.619141,36.547029,1.435996e+08,0.019797,38.591462,0.529786
std,11.002370,11.103250,10.925538,11.029897,11.259383,7.153802e+07,0.464672,10.990491,0.499310
min,22.500000,22.917500,22.367500,22.584999,20.883032,4.544800e+07,-2.275002,22.584999,0.000000
25%,28.950001,29.130624,28.627500,28.876250,26.493847,9.512290e+07,-0.209376,28.875000,0.000000
50%,37.593750,38.012501,37.293751,37.638750,35.679941,1.263050e+08,0.030001,37.637501,1.000000
75%,46.500627,46.845624,46.167500,46.500626,44.616409,1.714582e+08,0.261876,46.465000,1.000000
max,72.779999,73.492500,72.379997,73.412498,71.810936,6.488252e+08,2.217499,72.879997,1.000000


## 3. Sentimental Analysis using VADER
VADER (Valence Aware Dictionary and sEntiment Reasoner) is particularly good for analysing sentiment of social media texts as it takes into account linguistic features within a social media text. VADER provides 4 normalised sentiment scores: Negative, Neutral, Positive and Compound. The positive, negative, and neutral scores (ranging from 0 to 1) represent the proportion of the text that is positive, negative, or neutral, respectively. The compound score (ranging from -1 to 1) provides an indication as overall indication of the sentiment of the text.

Given the social nature of tweets, we will be using the VADER model as it is very appropriate.


In [14]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mingyang/nltk_data...


The dataset is very massive with over 1 million rows. Running the model will take a few days to complete which is not very efficient. We will concatenate tweets of the same day together. 

To show the concatenating tweets will not affect the conclusion we can derive from the score, let us take a look at the following examples. When we concatenate 2 strings with opposing sentiment, the negative and positive values are scaled closer to 0. Concatenating 2 strings with the same sentiment will result in an unchanged sentiment value. Furthermore, given that these tweets are related to AAPL stock price, concatenating tweets will not result in too much loss in information or context

In [15]:
sia.polarity_scores("I am happy")

{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'compound': 0.5719}

In [16]:
sia.polarity_scores("I am sad")

{'neg': 0.756, 'neu': 0.244, 'pos': 0.0, 'compound': -0.4767}

In [17]:
#Concatenating 2 strings with different sentiment 
sia.polarity_scores("I am happy I am sad")

{'neg': 0.352, 'neu': 0.227, 'pos': 0.42, 'compound': 0.1531}

In [18]:
#Concatenating 2 strings with same sentiment
sia.polarity_scores("I am happy I am happy")

{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'compound': 0.8126}

In [50]:
#Grouping tweets of the same day together
groupedtweet = tweet1.groupby('date')['Tweet'].apply(' '.join).reset_index()
groupedtweet

,date,Tweet
0,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! http..."
1,2015-01-02,Stock_Tracker: $AAPL Recent Headlines 5 New Ye...
2,2015-01-03,Free 5€ in account balance for first 100.000 m...
3,2015-01-04,Jake13: $AAPL Opinion Today Click here to find...
4,2015-01-05,Get $25 when you sign up http #Payoneer w/ my ...
...,...,...
1821,2019-12-27,$AAPL is starting to see Puts coming in. The 1...
1822,2019-12-28,$MTCH $DLTR $QCOM $INTC $AAPL $MSFT $SBUX $AMA...
1823,2019-12-29,Chart Patterns to Watch This Week 12-29-2019 h...
1824,2019-12-30,BOT $AAPL Jan 03 2020 W 287.50 Put @user $AAPL...


In [67]:
#Retrieving polarity scores
res = {}

for i,row in tqdm(groupedtweet.iterrows(), total=len(groupedtweet)):
    text = row['Tweet']
    res[i] = sia.polarity_scores(text)

  4%|▍         | 71/1826 [01:05<27:01,  1.08it/s]  


KeyboardInterrupt: 

In [75]:
# Adding polarity scores to dataframe
result = pd.DataFrame(res).T
tweet_sentiments = pd.merge(groupedtweet, result, left_index=True, right_index=True)
tweet_sentiments.set_index('date', inplace=True)

tweet_sentiments

,Tweet,neg,neu,pos,compound
date,,,,,
2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! http...",0.034,0.848,0.119,0.9999
2015-01-02,Stock_Tracker: $AAPL Recent Headlines 5 New Ye...,0.052,0.837,0.111,1.0000
2015-01-03,Free 5€ in account balance for first 100.000 m...,0.025,0.821,0.154,1.0000
2015-01-04,Jake13: $AAPL Opinion Today Click here to find...,0.030,0.828,0.142,1.0000
2015-01-05,Get $25 when you sign up http #Payoneer w/ my ...,0.046,0.824,0.129,1.0000
...,...,...,...,...,...
2015-03-08,Blackberry is back. RT “@ACInvestorBlog: $AAPL...,0.024,0.818,0.158,1.0000
2015-03-09,"Don't sell the news, buy it! My limit order ju...",0.044,0.830,0.126,1.0000
2015-03-10,#Apple & #AppleWatch: A New Era for #digitalhe...,0.052,0.822,0.126,1.0000


## Merging stockprice and tweet dataframe

In the process of merging, we will drop the tweets that were tweeted on Saturday, Sunday and public holidays as the stock market is not open on those days.

In [76]:
sentimentAndStockPrice = pd.merge(tweet_sentiments, df[['close','label','open','high','low','adjclose','volume']], left_index=True, right_index=True)
display(sentimentAndStockPrice)

/var/folders/hw/wb159grn5vqfnk074jgc32h00000gn/T/ipykernel_14516/841993234.py:1: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



,Tweet,neg,neu,pos,compound,close,label,open,high,low,adjclose,volume
date,,,,,,,,,,,,
2015-02-12,My brain is mixing $AAPL and AAP & ISIS and IS...,0.04,0.84,0.12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
